# 🚀 Automated LoRA Training Pipeline

This notebook automatically trains a LoRA adapter for your selected model using your custom dataset.

**Steps:**
1. Install required packages
2. Load your dataset from the backend
3. Configure LoRA parameters
4. Train the model
5. Save and upload the trained adapter

In [ ]:
# Install required packages
!pip install -q transformers datasets peft accelerate bitsandbytes trl einops requests

In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime

# Parse URL parameters
import sys
from urllib.parse import urlparse, parse_qs

# Get job_id and model from URL (will be passed as parameters)
# For manual testing, set these values:
JOB_ID = "test-job-123"  # Will be replaced by actual job ID from URL
MODEL_NAME = "llama-2-7b"  # Will be replaced by selected model
API_URL = "https://slmllm-backend.vercel.app"

print(f"Job ID: {JOB_ID}")
print(f"Model: {MODEL_NAME}")
print(f"API URL: {API_URL}")

In [ ]:
# Model mapping
MODEL_MAP = {
    "llama-2-7b": "meta-llama/Llama-2-7b-hf",
    "llama-2-13b": "meta-llama/Llama-2-13b-hf",
    "mistral-7b": "mistralai/Mistral-7B-v0.1",
    "phi-2": "microsoft/phi-2",
    "gemma-7b": "google/gemma-7b",
    "tinyllama-1.1b": "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
}

model_id = MODEL_MAP.get(MODEL_NAME, "TinyLlama/TinyLlama-1.1B-Chat-v1.0")
print(f"Using model: {model_id}")

In [ ]:
# Function to update training status
def update_status(status, progress=0, error=None):
    try:
        data = {
            "status": status,
            "progress": progress
        }
        if error:
            data["error"] = error
        
        response = requests.post(
            f"{API_URL}/api/train/update/{JOB_ID}",
            data=data
        )
        print(f"Status update: {status} ({progress}%)")
    except Exception as e:
        print(f"Failed to update status: {e}")

# Update status to training
update_status("training", 5)

In [ ]:
# Load dataset (simulated - in production, fetch from backend)
# For now, create a sample dataset
sample_data = {
    "input": [
        "What is machine learning?",
        "Explain neural networks",
        "What is deep learning?"
    ],
    "output": [
        "Machine learning is a subset of AI that enables systems to learn from data.",
        "Neural networks are computational models inspired by the human brain.",
        "Deep learning is a subset of machine learning using multi-layered neural networks."
    ]
}

df = pd.DataFrame(sample_data)
print(f"Loaded {len(df)} training examples")
print(df.head())

update_status("training", 10)

In [ ]:
# Import training libraries
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import Dataset

update_status("training", 15)

In [ ]:
# Configure 4-bit quantization for efficient training
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
print(f"Loading {model_id}...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("Model loaded successfully!")
update_status("training", 25)

In [ ]:
# Configure LoRA
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")

update_status("training", 30)

In [ ]:
# Prepare dataset for training
def format_instruction(sample):
    return f"### Input:\n{sample['input']}\n\n### Output:\n{sample['output']}"

# Convert to HuggingFace dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.map(lambda x: {"text": format_instruction(x)})

print(f"Dataset prepared with {len(dataset)} examples")
update_status("training", 35)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./lora_output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=2,
    logging_steps=10,
    save_steps=50,
    warmup_steps=10,
    max_grad_norm=0.3,
    group_by_length=True,
    lr_scheduler_type="cosine",
)

# Create trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    dataset_text_field="text",
    max_seq_length=512,
)

print("Trainer configured successfully!")
update_status("training", 40)

In [ ]:
# Train the model
print("🚀 Starting training...")
update_status("training", 45)

try:
    trainer.train()
    print("✅ Training completed successfully!")
    update_status("training", 90)
except Exception as e:
    print(f"❌ Training failed: {e}")
    update_status("failed", 0, str(e))
    raise

In [ ]:
# Save the trained model
output_dir = f"./lora_model_{JOB_ID}"
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")
update_status("training", 95)

In [ ]:
# Test the trained model
print("\n🧪 Testing trained model...")
test_prompt = "What is machine learning?"
inputs = tokenizer(format_instruction({"input": test_prompt, "output": ""}), return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"\nPrompt: {test_prompt}")
print(f"Response: {response}")

In [ ]:
# Complete the training job
update_status("completed", 100)
print("\n✅ Training pipeline completed successfully!")
print(f"\n📦 Your trained LoRA adapter is ready in: {output_dir}")
print("\n💡 To use this model:")
print("1. Download the adapter files from this Colab")
print("2. Load it with PEFT in your application")
print("3. Merge with base model or use directly")